## Comparing the performance of Gated Recurrent Units With and Without Attention
By learning to predict the text of H.G. Wells' The Time Machine

### Dataset

In [1]:
raw_text = open('../datasets/timemachine.txt').read()

### Let's tokenize the dataset


In [2]:
import re
from transformers import AutoTokenizer

# Remove punctuation and convert to lowercase
text = raw_text.lower()
text = re.sub(r'[^a-z]+', ' ', text)

# Tokenize into characters
char_tokenized_text = list(text)

/Users/vikram/dev/xjax/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
",".join(char_tokenized_text[200:300])

's,e, ,i,t, ,u,n,d,e,r, ,t,h,e, ,t,e,r,m,s, ,o,f, ,t,h,e, ,p,r,o,j,e,c,t, ,g,u,t,e,n,b,e,r,g, ,l,i,c,e,n,s,e, ,i,n,c,l,u,d,e,d, ,w,i,t,h, ,t,h,i,s, ,e,b,o,o,k, ,o,r, ,o,n,l,i,n,e, ,a,t, ,w,w,w, ,g,u,t'

In [4]:
# Most common tokens
from collections import Counter
# Flatten the list of tokens
# Count the tokens
token_counts = Counter(char_tokenized_text)
print(f"Total tokens: {len(token_counts)}")
# Most common tokens
most_common_tokens = token_counts.most_common(10)
for token, count in most_common_tokens:
    print(f"{token} {count}")

Total tokens: 27
  35850
e 19667
t 15040
a 12700
i 11254
o 11082
n 10943
s 9242
r 8833
h 8786


In [5]:
MIN_FREQ = 1 ## Minimum token frequency to include in the vocab

### Generate the Vocabulary
tok_to_idx = {}
idx_to_tok =  list(sorted(set(['<unk>'] + [tok for tok,count in token_counts.items() if count >= MIN_FREQ])))
tok_to_idx = {tok: idx for idx, tok in enumerate(idx_to_tok)}


# Vocabulary size
vocab_size = len(idx_to_tok)
print(f"Vocabulary size: {vocab_size}")



Vocabulary size: 28


In [6]:
import jax.numpy as jnp

def sliding_window(seq, window_size, overlap):
    for i in range(0, len(seq) - window_size, window_size - overlap):
        yield [ tok_to_idx[tok] if tok in tok_to_idx else tok_to_idx['<unk>'] for tok in seq[i:i + window_size] ]


## Generate dataset 
def generate_data(text, seq_length, overlap):
    num_tokens = len(char_tokenized_text)
    return jnp.array([ seq for seq in sliding_window(char_tokenized_text, seq_length, overlap)])

In [7]:
SEQUENCE_LENGTH = 32
WINDOW_OVERLAP = 0

data = generate_data(text, SEQUENCE_LENGTH, WINDOW_OVERLAP)
X = data[:,:-1]
Y = data[:,1:]

print(X.shape, Y.shape)
print([idx_to_tok[i] for i in X[0]])
print([idx_to_tok[i] for i in Y[0]])


(5991, 31) (5991, 31)
['p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 's', ' ', 't', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a']
['r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 's', ' ', 't', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c']


In [13]:
import logging

from blinker import signal

import jax

from xjax.models import gru
from xjax.signals import train_epoch_started, train_epoch_completed

rng = jax.random.key(42)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

HIDDEN_SIZE=32
EPOCHS=30
LEARNING_RATE = 10**(-2)
MAX_GRAD = 2

params, model = gru.gru(rng, vocab_size=vocab_size, hidden_size=HIDDEN_SIZE)

params[0].shape

@train_epoch_started.connect_via(model)
def collect_events(_, *, epoch, elapsed, **__):
    logger.info(f"Started epoch={epoch}, elapsed={elapsed:0.4f}s")

@train_epoch_completed.connect_via(model)
def collect_events(_, *, epoch, loss, elapsed, **__):
    logger.info(f"Completed epoch={epoch}, loss={loss:0.4f}, elapsed={elapsed:0.4f}s")


# I train a GRU model on the data 
trained_params = gru.train(model, rng=rng, params=params, 
                                            X=X, Y=Y, 
                                            vocab_size=vocab_size, 
                                            batch_size=1024,
                                            num_epochs=EPOCHS, 
                                            learning_rate=LEARNING_RATE,
                                            max_grad=MAX_GRAD)



INFO:__main__:Started epoch=0, elapsed=0.0022s
INFO:__main__:Completed epoch=0, loss=659.7244, elapsed=14.6347s
INFO:__main__:Started epoch=1, elapsed=14.6351s
INFO:__main__:Completed epoch=1, loss=568.9825, elapsed=20.3553s
INFO:__main__:Started epoch=2, elapsed=20.3558s
INFO:__main__:Completed epoch=2, loss=554.8980, elapsed=26.1008s
INFO:__main__:Started epoch=3, elapsed=26.1016s
INFO:__main__:Completed epoch=3, loss=546.5063, elapsed=32.2924s
INFO:__main__:Started epoch=4, elapsed=32.2928s
INFO:__main__:Completed epoch=4, loss=543.5025, elapsed=38.1424s
INFO:__main__:Started epoch=5, elapsed=38.1430s
INFO:__main__:Completed epoch=5, loss=540.0598, elapsed=44.1694s
INFO:__main__:Started epoch=6, elapsed=44.1698s
INFO:__main__:Completed epoch=6, loss=536.1841, elapsed=49.9213s
INFO:__main__:Started epoch=7, elapsed=49.9216s
INFO:__main__:Completed epoch=7, loss=536.1277, elapsed=55.6844s
INFO:__main__:Started epoch=8, elapsed=55.6848s
INFO:__main__:Completed epoch=8, loss=533.6682, e

In [14]:
import xjax

# I generate sequences from a prefix
prefix_str = "p"
prefix = [tok_to_idx[c] for c in prefix_str]
generated = []
for i in range(2):
    rng, sub_rng = jax.random.split(rng)
    y = xjax.models.gru.generate(rng=sub_rng, prefix= prefix, params=trained_params, hidden_size=HIDDEN_SIZE, vocab_size=vocab_size, max_len=200) 
    generated.append("".join([idx_to_tok[i] for i in y]))



In [15]:
generated[0]

'pales lound about laments gated the quered had uppond the con the confusts just more the reece that the time at fy first were to sto of the more of the saw the rewougher were oull this in be hed of se'